Toy GROTH16 implementation 

In [7]:
import numpy as np
import galois
from functools import reduce
from py_ecc.bn128 import G1, G2, multiply, add, curve_order, pairing, neg, final_exponentiate, FQ12, Z1

GF = galois.GF(curve_order)